# 1. Kick-off: Libraries Importing, Variables Setup and Functions

### 1.1 Libraries and Variables setup

In [ ]:
import numpy as np
import pandas as pd
import copy
import scipy
from scipy import stats
from scipy.stats import gaussian_kde
from fitter import Fitter
import math
import random
import time
import datetime as dt
from datetime import datetime

import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.set_config_file(offline=True, dimensions=(800,500))

from IPython.display import display, HTML, Javascript
from src.bloomberg_template import set_bloomberg_dark, set_bloomberg_light, get_color, get_colorscale

import warnings
warnings.simplefilter("ignore")

Setting the parameters:

In [ ]:
# Medimos el tiempo de ejecucion del programa
star_time = time.time()

MARKET_DAYS_YEAR = 252
OUTPUT_NAME_1 = 'historical-Adj_prices-byma'
OUTPUT_NAME_2 = 'historical-Adj_prices_plus-byma'
EXPORT_DATA = False

TICKERS = ['Index', 'BBAR', 'BMA', 'GGAL', 'SUPV', 'VALO']
TICKERS_YF = ['^MERV', 'BBAR.BA', 'BMA.BA', 'GGAL.BA', 'SUPV.BA', 'VALO.BA']

START_DATE = '2023-12-11'
END_DATE = ''

### 1.2 Functions

In [ ]:
def get_data(df, categoria, ticker):
	return df.loc[:, (categoria, ticker)]


text_custom = f"""
"""
print(text_custom.lstrip())


def descargar_datos_yf(tickers, start_date=None, end_date=None, delay=2):
	if start_date is None:
		start_date = dt.datetime(2015, 1, 1)
	if end_date is None:
		end_date = dt.datetime.now()

	data_dict = {}
	for ticker in tickers:
		try:
			df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
			if not df.empty:
				data_dict[ticker] = df
				print(f'Descargado: {ticker}')
			else:
				print(f'Sin datos: {ticker}')
		except Exception as e:
			print(f'Error descargando {ticker}: {e}')
		time.sleep(delay)
	
	if data_dict:
		df = pd.concat(data_dict, axis=1)
	else:
		df = pd.DataFrame()

	return df

In [ ]:
def insert_ohlcav_cols(instancia):
	"""
	Inserta las columnas de precio OHLCAV a cada DataFrame vacio dentro de la clase TickerData.
	"""
	tickers_added = ''
	for ticker in instancia.tickers:
		df = getattr(instancia, ticker)
		try:
			df['Open'] = instancia.prices.loc[:, (ticker, tipo_precio[4])].squeeze()
			df['High'] = instancia.prices.loc[:, (ticker, tipo_precio[2])].squeeze()
			df['Low'] = instancia.prices.loc[:, (ticker, tipo_precio[3])].squeeze()
			df['Close'] = instancia.prices.loc[:, (ticker, tipo_precio[1])].squeeze()
			df['Adj Close'] = instancia.prices.loc[:, (ticker, tipo_precio[0])].squeeze()
			df['Volume'] = instancia.prices.loc[:, (ticker, tipo_precio[5])].squeeze()
			df.fillna(0, inplace=True)
			df[df < 0] = 0
			tickers_added += ticker + ', '
		except KeyError as e:
			print(f'Error al agregar precios para {ticker}: {e}')

	print(f'Se insertaron las columnas OHLCAV en los DF {tickers_added.rstrip(", ")}.')


In [ ]:
def calc_prices_df(self, column_price):
	""" Crea un nuevo DF solo con un precio específico de todos los activos """

	prices_dict = {}
	tickers_added = []

	for ticker in self.tickers:
		# Trabajamos sobre una copia
		df = getattr(self, ticker).copy(deep=True)
		if column_price not in df.columns:
			raise KeyError(f'La columna {column_price} no existe en el DF {ticker}')
				
		price_values = df[column_price]
		
		prices_dict[ticker] = price_values
		tickers_added.append(ticker)
	
	prices_df = pd.DataFrame(prices_dict)
	print(f'El DF de {column_price}, fue creado con los activos {", ".join(tickers_added)}.')
	return prices_df


def calc_returns_df(self, method, column_price):
	""" Crea un nuevo DF con los returns de todos los activos """
	valid_methods = ['log', 'simple']
	if method not in valid_methods:
		raise ValueError(f'El método no es válido. Usar uno de: {", ".join(valid_methods)}')

	returns_dict = {}
	tickers_added = []

	for ticker in self.tickers:
		# Trabajamos sobre una copia
		df = getattr(self, ticker).copy(deep=True)
		if column_price not in df.columns:
			raise KeyError(f'La columna {column_price} no existe en el DF {ticker}')
		
		if method == 'log':
			return_values = np.log(df[column_price]).diff().fillna(0)
		else:
			return_values = df[column_price].pct_change().fillna(0)
		
		returns_dict[ticker] = return_values
		tickers_added.append(ticker)
	
	returns_df = pd.DataFrame(returns_dict)
	print(f'El DF de {method.capitalize()} Returns en función de {column_price}, fue creado con los activos {", ".join(tickers_added)}.')
	return returns_df


def calc_returns_volat_df(self, method, column_price, window=40):
	""" Creamos un nuevo DF con returns y volatilidad de todos los activos """
	valid_methods = ['log', 'simple']
	if method not in valid_methods:
		raise ValueError(f'El método no es válido. Usar uno de: {", ".join(valid_methods)}')
	
	returns_volat_dict = {}
	tickers_added = []

	for ticker in self.tickers:
		# Trabajamos sobre una copia
		df = getattr(self, ticker).copy(deep=True)
		if column_price not in df.columns:
			raise KeyError(f'La columna {column_price} no existe en el DF {ticker}')

		if method == 'log':
			return_values = np.log(df[column_price]).diff().fillna(0)
		else:
			return_values = df[column_price].pct_change().fillna(0)
		
		volat_values = (df['returns'].rolling(window=window).std().fillna(0)) * np.sqrt(MARKET_DAYS_YEAR)

		returns_volat_dict[ticker + '_returns'] = return_values
		returns_volat_dict[ticker + '_volat_' + str(window)] = volat_values
		tickers_added.append(ticker)

	returns_volat_df = pd.DataFrame(returns_volat_dict)
	print(f'Se creo el DF de {method.capitalize()} Returns en función de {column_price} y Volatilidad Anual con ventana de {window} ruedas.\nContiene los activos {", ".join(tickers_added)}.')
	return returns_volat_df


In [ ]:
def calculate_returns(df, method='log', column_price='Adj Close'):
	""" 
	Calcula los returns en funcion del metodo y precio. 
	Parámetros:
	- df: DataFrame original.
	"""
	valid_methods = ['log', 'simple']
	if method not in valid_methods:
		raise ValueError(f'El método no es válido. Usar uno de: {", ".join(valid_methods)}')

	if column_price not in df.columns:
		raise KeyError(f'La columna {column_price} no existe en el DF {ticker}')

	if method == 'log':
		df['returns'] = np.log(df[column_price]).diff().fillna(0)
	else:
		df['returns'] = df[column_price].pct_change().fillna(0)

	msg = f'{method.capitalize()} Returns sobre {column_price} en "returns"'

	return df, msg


def calculate_volat(df, method='log', column_price='Adj Close', window=40):
	""" 
	Calcula la volatilidad en funcion del metodo y precio. 
	Parámetros:
	- df: DataFrame original.
	"""
	valid_methods = ['log', 'simple']
	if method not in valid_methods:
		raise ValueError(f'El método no es válido. Usar uno de: {", ".join(valid_methods)}')

	if column_price not in df.columns:
		raise KeyError(f'La columna {column_price} no existe en el DF {ticker}')
	
	if method == 'log':
		returns = np.log(df[column_price]).diff().fillna(0)
	else:
		returns = df[column_price].pct_change().fillna(0)
	
	df[f'volat_{window}'] = (returns.rolling(window=window).std().fillna(0)) * np.sqrt(MARKET_DAYS_YEAR)

	msg = f'Volatilidad Anual con ventana de {window} ruedas en función de {method.capitalize()} Returns sobre {column_price} en "{"volat_" + str(window)}"'

	return df, msg


### 1.3 Classes

#### class TickerData()

Necesitamos que este notebook sea escalable, por lo que vamos a crear una ***Class*** para poder crear tantos DF como Tickers ingresemos.

Creamos una Class que al instanciarla reliza lo siguiente:
*	Crea un DF sin datos por cada activo especificado en la lista `tickers`. Se accede a cada uno de ellos por medio de `class_instance.ticker`.

*	Trata los missing values y los valores negativos.

*	`class_instance.list_tickers()`: Muestra los tickers y DF disponibles.

*	`class_instance.load_ohlcav()`: Inserta las columnas de precio OHLCAV a cada DF vacio dentro de la Class.

*	`class_instance.create_returns_df()`: Crea un nuevo DF solo con los returns de todos los activos.

*	`class_instance.create_returns_volat_df(window=40)`: Crea un nuevo DF solo con los returns y volatilidad de todos los activos.
	
*	`class_instance.add_columns(function, **kwargs)`: Agrega columnas al DF de cada activo llamando a una Function externa. Esto nos posibilita utilizar Functions fuera de la Class para agregar comportamientos a la Class y que se apliquen a todos los DF (returns, medias, indicadores, etc).

*	`class_instance.backup_dataframes(self, suffix='_v1')`: Crea backups de los DF de la clase con un sufijo: "data.BBAR_v1".

*	`class_instance.backup_dataframes_to_globals(self, suffix='_v1')`: Crea backups de los DF de la clase como variables GLOBALES, con un sufijo: "BBAR_v1".

In [ ]:
class TickerData:
	def __init__(self, tickers, prices, tipo_precio=None):
		self.tickers = tickers
		self.prices = prices

		tickers_added = ''
		for ticker in tickers:
			df = pd.DataFrame({}, index=prices.index)			
			df.fillna(0, inplace=True)
			df[df < 0] = 0
			setattr(self, ticker, df)	# asignamos el DF como atributo de la Class
			tickers_added += ticker + ', '
		
		print(f'Se agregaron a la Class los DataFrame de {tickers_added.rstrip(", ")}.')


	def list_tickers(self):
		""" Vemos los DF creados, correspondientes a cada ticker """
		return self.tickers


	def load_ohlcav(self):
		""" La Function externa inserta columnas de precio OHLCAV en cada DF VACIO de la clase. """
		insert_ohlcav_cols(self)	


	def add_columns(self, function, **kwargs):
		"""
		Agregamos columnas a todos los DF de la Class llamando a una Function externa.

		Parámetros:
		- function: la function externa que recibe un DF y devuelve un DF modificado con nuevas cols o calculos.
		- kwargs: parámetros adicionales que pueda necesitar la function externa.

		Ejemplo de uso: data.add_columns(calculate_ema, EMA1=10, EMA2=20)
		"""
		for ticker in self.tickers:
			df = getattr(self, ticker)
			df, msg = function(df, **kwargs)
			setattr(self, ticker, df)
		
		print(f'Agregados a los DF de la Class usando "{function.__name__}": {msg}.')
		

	def create_prices_df(self, column_price='Close'):
		self.prices_df = calc_prices_df(self, column_price=column_price)
		return self.prices_df
		

	def create_returns_df(self, method='log', column_price='Adj Close'):
		self.returns_df = calc_returns_df(self, method=method, column_price=column_price)
		return self.returns_df
	

	def create_returns_volat_df(self, method='log', column_price='Adj Close', window=40): 
		self.returns_volat_df = calc_returns_volat_df(self, method=method, column_price=column_price, window=window)
		return self.returns_volat_df
	

	def backup_dataframes(self, suffix='_v1'):
		df_added = []
		for ticker in self.tickers:
			df = getattr(self, ticker)
			df_copy = df.copy(deep=True)
			setattr(self, f'{ticker}{suffix}', df_copy)
			df_added.append(f'{ticker}{suffix}')
		print(f'Se crearon copias INTERNAS de la INSTANCIA de los DF:\n{", ".join(df_added)}.\n')


	def backup_dataframes_to_globals(self, suffix='_v1'):
		df_added = []
		for ticker in self.tickers:
			df = getattr(self, ticker)
			df_copy = df.copy(deep=True)
			globals()[f'{ticker}{suffix}'] = df_copy
			# setattr(self, f'{ticker}{suffix}', df_copy)
			df_added.append(f'{ticker}{suffix}')
		print(f'Se crearon copias GLOBALES de los DF:\n{", ".join(df_added)}.\n')


# 2. Data Loading

In [ ]:
tickers = TICKERS
tickers_yf = TICKERS_YF

start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.now() if not END_DATE else datetime.strptime(END_DATE, "%Y-%m-%d")

raw_data = descargar_datos_yf(tickers_yf, start_date, end_date)
prices = raw_data.copy()

prices.index = prices.index.strftime('%Y-%m-%d')
prices.index = pd.to_datetime(prices.index)

print('Index type:', prices.index.dtype)
prices.head()

# 3. Data Cleaning

En esta etapa realizaremos lo siguiente:

1.	Modificamos los nombres de los Tickers, para mayor comodidad.

2.	Creamos DF limpios de todos los activos con precios OHLCAV.

3.	Hacemos un Checkpoint y guardamos lo realizado hasta el momento.

In [ ]:
#prices = raw_data.copy()

Cambiamos los nombres de los activos.

In [ ]:
for ticker_y, name in zip(tickers_yf, tickers):
	prices.rename(columns={ticker_y: name}, inplace=True)

df = prices
print("\nNiveles del índice en columnas:")
for i, name in enumerate(df.columns.names):
	print(f"Nivel {i} ({name}): {df.columns.unique(level=i).tolist()}")

prices.head()

In [ ]:
tipo_precio = df.columns.unique(level=1).tolist()
tipo_precio

Creamos DF limpios de todos los activos con precios OHLCAV:

In [ ]:
data = TickerData(tickers, prices, tipo_precio)
data.load_ohlcav()

# 4. Data Transformation

### Dataframe de cada activo

Calculamos y agregamos a cada DF de los activos, lo siguiente:

*	Retornos logaritmicos
*	Volatilidad de las ultimas 40 ruedas, anualizada.

In [ ]:
# Agregamos los retornos logaritmicos
data.add_columns(calculate_returns, method='log', column_price='Adj Close')

# Agregamos la volatilidad anual
data.add_columns(calculate_volat, method='log', column_price='Adj Close', window=40)


In [ ]:
data.GGAL.head()

Creamos un Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifican con el sufijo *"v1"*:

In [ ]:
# Checkpoint
data.backup_dataframes(suffix='_v1')
data.backup_dataframes_to_globals(suffix='_v1')

Seguimos trabajando sobre los DF originales, manteniendo una copia de los mismos, en caso de necesitar revertir cambios realizados.

### Dataframe de Precios

Creamos un DF con los Precios de todos los activos (Open, Low, High, Close o Adj Close).

In [ ]:
prices = data.create_prices_df(column_price='Adj Close')
prices.head()

Creamos otro Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifica con el sufijo *"v1"*:

In [ ]:
# Checkpoint
df = prices
df_name = 'prices'

df = globals()[df_name]
globals()[f'{df_name}_v1'] = df.copy(deep=True)

### Dataframe de Retornos

Creamos un DF con los Retornos de todos los activos.

In [ ]:
returns = data.create_returns_df(method='log', column_price='Adj Close')
returns.head()

Creamos otro Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifica con el sufijo *"v1"*:

In [ ]:
# Checkpoint
df = returns
df_name = 'returns'

df = globals()[df_name]
globals()[f'{df_name}_v1'] = df.copy(deep=True)

### Dataframe de Retornos y Volatilidad

Creamos un DF con los Retornos y Volatilidad de todos los activos.

In [ ]:
returns_volat = data.create_returns_volat_df(method='log', column_price='Adj Close', window=40)
returns_volat.head()

Creamos otro Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifica con el sufijo *"v1"*:

In [ ]:
# Checkpoint
df = returns_volat
df_name = 'returns_volat'

df = globals()[df_name]
globals()[f'{df_name}_v1'] = df.copy(deep=True)

### Cálculos estadísticos

Creamos una matriz de correlación de los retornos.

In [ ]:
correlations = returns.corr().round(2)
correlations

Calculamos algunos estadísticos.

In [ ]:
datos = returns

mu = np.mean(datos, axis=0)
sigma = np.std(datos, ddof=1, axis=0)

In [ ]:
mu

In [ ]:
sigma

In [ ]:
returns_prom = returns.mean() * MARKET_DAYS_YEAR
returns_prom

# 5. Visualization

In [ ]:
prices_norm = prices.normalize()
tickers

## 5.1 Plot normalizado

In [ ]:
datos = prices_norm
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]

periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
graf_titulo = f'Precios Normalizados de principales activos bancarios | Período {periodo}'
eje_x_titulo = f'Fechas respecto de cotizaciones diarias'
eje_y_titulo = f'Variación base 100 respecto al inicio de la serie (normalizado)'

eje_x = datos.index
dias_total = (fecha_fin - fecha_inicio).days
dias_extra = int(dias_total * 0.01)  # 1% del rango
x_min_extendido = fecha_inicio - pd.Timedelta(days=dias_extra)
x_max_extendido = fecha_fin + pd.Timedelta(days=dias_extra)

tickvals = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS').to_list()
if fecha_inicio not in tickvals: tickvals.insert(0, fecha_inicio)
if fecha_fin not in tickvals: tickvals.append(fecha_fin)

# Creamos linea vertical de inicio y fin    
linea_vertical = lambda fecha, color: dict(
	type="line", xref="x", yref="paper", x0=fecha, x1=fecha, y0=0, y1=1, line=dict(color=color, width=1.0, dash="dash"))

line_props = [
	{'y': datos[tickers[0]], 'color': get_color(7), 'width': 2, 'text_leg': f'{tickers[0]}'},
	{"y": datos[tickers[1]], 'color': get_color(0), 'width': 1.5, 'text_leg': f'{tickers[1]}'},
	{"y": datos[tickers[2]], 'color': get_color(1), 'width': 1.5, "text_leg": f'{tickers[2]}'},
	{"y": datos[tickers[3]], 'color': get_color(2), 'width': 1.5, "text_leg": f'{tickers[3]}'},
	{"y": datos[tickers[4]], 'color': get_color(3), 'width': 1.5, "text_leg": f'{tickers[4]}'},
	{"y": datos[tickers[5]], 'color': get_color(4), 'width': 1.5, "text_leg": f'{tickers[5]}'},
]

fig_activos_norm = go.Figure()

for i, prop in enumerate(line_props):
	fig_activos_norm.add_trace(go.Scatter(
		x=eje_x, y=prop['y'],
		mode='lines',
		line=dict(width=prop['width'], color=prop['color']), opacity=0.8,
		name=prop['text_leg'], showlegend=True,
		yaxis='y2' if i == 0 else 'y',
		fillcolor='rgba(0, 0, 255, 0.2)' if i == 0 else None, 
		fill='tozeroy' if i == 0 else None,))
	
	if i == 0:
		fig_activos_norm.add_shape(
			type="line", x0=min(eje_x), x1=max(eje_x), 
			y0=prop['y'].iloc[-1], y1=prop['y'].iloc[-1],
			line=dict(color=get_color(4), width=1, dash="dot"))

fig_activos_norm.add_shape(
	linea_vertical(fecha_inicio, get_color(2)))
fig_activos_norm.add_shape(
	linea_vertical(fecha_fin, get_color(2)))

fig_activos_norm.update_layout(
	width=900, height=600, margin=dict(l=80, r=70, t=80, b=100),
	title=dict(text=graf_titulo, x=0.5, y=0.95),
	xaxis=dict(
		title=dict(text=eje_x_titulo, standoff=20), type='date', tickvals=tickvals, tickformat='%Y-%m-%d', tickangle=45, range=[x_min_extendido, x_max_extendido],# dtick='M1',
	),
	yaxis=dict(
		title=dict(text=eje_y_titulo, standoff=20),	type='linear', side='left'),
	yaxis2=dict(
		overlaying='y', side='right', showgrid=False, showticklabels=True, matches='y'),
	legend=dict(bordercolor="black", borderwidth=1, x=0.02, y=0.99, xanchor="left", yanchor="top"),
)

set_bloomberg_dark(fig_activos_norm)
fig_activos_norm.show()


## 5.2 Plot Normalized Prices, Returns & Volatility

In [ ]:
def add_vertical_axis(fig, assets_graph, cols_graph, title_primary=None, color_primary=None,
                              title_secondary=None, color_secondary=None):
    """
    Agrega modificaciones a ejes primarios y secundarios en subplots de Plotly, en un solo update_layout.

    Parámetros:
    -----------
    fig : go.Figure
        Figura de Plotly ya creada con make_subplots.
    assets_graph : int
        Número de subplots (activos gráficos).
    cols_graph : int
        Número de columnas del layout de subplots.
    title_primary : str, opcional
        Título del eje primario (solo en primera columna).
    color_primary : str, opcional
        Color de los ticks del eje primario.
    title_secondary : str, opcional
        Título del eje secundario (solo en segunda columna).
    color_secondary : str, opcional
        Color de los ticks del eje secundario.
    """
    layout_updates = {}

    for i in range(assets_graph):
        if i == 0:
            eje_primario = 'yaxis'
            eje_secundario = 'yaxis2'
            overlaying = 'y'
        else:
            eje_primario = f'yaxis{1 + i*2}'
            eje_secundario = f'yaxis{2 + i*2}'
            overlaying = f'y{1 + i*2}'

        col_subplot = (i % cols_graph) + 1
        
        # Eje primario
        layout_updates[eje_primario] = dict(
            showgrid=True, zeroline=False, tickfont=dict(color=color_primary),)
        if title_primary and col_subplot == 1:
            layout_updates[eje_primario]['title'] = dict(text=title_primary, standoff=20)

        # Eje secundario
        layout_updates[eje_secundario] = dict(
            overlaying=overlaying, side='right', showgrid=False, zeroline=False, tickformat=".0%", tickfont=dict(color=color_secondary),)
        if col_subplot == 2:
            layout_updates[eje_secundario]['title'] = dict(text=title_secondary, standoff=20)

    fig.update_layout(**layout_updates)

In [ ]:
datos = prices_norm
# fecha_inicio = datos.index[0]
# fecha_fin = datos.index[-1]
fecha_inicio, fecha_fin = datos.index[[0, -1]]

periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
graf_titulo = f'Precios Norm, Rendimientos y Volatilidades de principales activos bancarios | Período {periodo}'
eje_x_titulo = f'Fechas respecto de cotizaciones diarias'
eje_y_titulo_1 = f'Variacion base 100 respecto al inicio de la serie'
eje_y_titulo_2 = f'Retorno y Volatilidad'

eje_x = datos.index
dias_total = (fecha_fin - fecha_inicio).days
dias_extra = int(dias_total * 0.01)
x_min_extendido = fecha_inicio - pd.Timedelta(days=dias_extra)
x_max_extendido = fecha_fin + pd.Timedelta(days=dias_extra)

tickvals = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='2MS').to_list()
if fecha_inicio not in tickvals: tickvals.insert(0, fecha_inicio)
if fecha_fin not in tickvals: tickvals.append(fecha_fin)

assets_graph = len(datos.columns)
rows_graph = (assets_graph // 2) + (assets_graph % 2)
cols_graph = 2


fig_volat = make_subplots(
	rows=rows_graph, cols=cols_graph,
	specs=[[{"secondary_y": True}, {"secondary_y": True}],
		[{"secondary_y": True}, {"secondary_y": True}],
		[{"secondary_y": True}, {"secondary_y": True}]],
	shared_xaxes=False, shared_yaxes=True, vertical_spacing=0.09, horizontal_spacing=0.10,
	subplot_titles=[f"<b>{col}</b>" for col in datos.columns]
)

for i, col in enumerate(datos.columns):
	rows_subplot = (i // cols_graph) + 1
	cols_subplot = (i % cols_graph) + 1

	data_price = datos[col].to_numpy()
	data_returns = returns_volat.iloc[:, 2*i].to_numpy()
	data_volat = returns_volat.iloc[:, 2*i+1].to_numpy()

	subplot_idx = i + 1  # Subplots empiezan en 1

	# 1) Precio normalizado - eje principal
	fig_volat.add_trace(go.Scatter(
		x=eje_x, y=data_price, mode='lines', line=dict(color=get_color(7), width=2),
		fillcolor='rgba(0, 0, 255, 0.2)', fill='tozeroy',	# Creamos la sombra
		name=f'Price Norm {col}', showlegend=False
	), row=rows_subplot, col=cols_subplot, secondary_y=False)

	# 2) Returns - eje secundario
	fig_volat.add_trace(go.Scatter(
		x=eje_x, y=data_returns, mode='lines', line=dict(color=get_color(1), width=1.0),
		name=f'Returns {col}', showlegend=False, yaxis=f'y{subplot_idx*2}'
	), row=rows_subplot, col=cols_subplot, secondary_y=True)

	# 3) Volatilidad - eje secundario
	fig_volat.add_trace(go.Scatter(
		x=eje_x, y=data_volat, mode='lines', line=dict(color=get_color(3), width=1.5),
		name=f'Volat_40 {col}', showlegend=False, yaxis=f'y{subplot_idx*2}'
	), row=rows_subplot, col=cols_subplot, secondary_y=True)

	# Agregamos linea de ultimo valor de Price
	fig_volat.add_shape(
		type="line", x0=min(eje_x), x1=max(eje_x), y0=data_price[-1], y1=data_price[-1],
		line=dict(color=get_color(4), width=1, dash="dot"
	), row=rows_subplot, col=cols_subplot)

	# Ejes X
	fig_volat.update_xaxes(
		type='date', tickvals=tickvals, tickformat='%Y-%m-%d', tickangle=45,
		range=[x_min_extendido, x_max_extendido], row=rows_subplot, col=cols_subplot)
	
	# Ejes Y primario
	fig_volat.update_yaxes(
		showticklabels=True, row=rows_subplot, col=cols_subplot, side='left')


# Anotaciones como leyendas
max_y_value = max([max(trace['y']) for trace in fig_volat.data if isinstance(trace['y'], (list, np.ndarray))])
x_pos = min(eje_x)
y_max = max_y_value

for i, col in enumerate(datos.columns):
	rows_subplot = (i // cols_graph) + 1
	cols_subplot = (i % cols_graph) + 1
	subplot_idx = i + 1  # Subplots empiezan en 1
	fig_volat.add_annotation(
		xref=f'x{2*subplot_idx-1}', yref=f'y{2*subplot_idx-1}', x=x_pos, y=y_max*0.99,
		xanchor="left", yanchor="top",
		text=(
			f"<b><span style='color:{get_color(7)};'>-- Price Norm</span><br>"
			f"<span style='color:{get_color(1)};'>-- Log Returns %</span><br>"
			f"<span style='color:{get_color(3)};'>-- Volat 40d %</span></b>"),
		align='left', font=dict(size=12), showarrow=False, 
		bordercolor="rgba(50,50,50,0.6)", borderwidth=0.8, borderpad=3, opacity=0.92,
		bgcolor="rgba(200, 200, 200, 0.20)", row=rows_subplot, col=cols_subplot)

# Títulos de ejes X
for col_id in range(1, cols_graph + 1):
	fig_volat.update_xaxes(title=dict(text=eje_x_titulo, standoff=20), row=rows_graph, col=col_id)

fig_volat.update_layout(
	width=1100, height=465*rows_graph, margin=dict(l=80, r=80, t=100, b=100), autosize=True, title=dict(text=graf_titulo, x=0.5, y=0.98))

# Agregamos las etiquetas y titulos primarios y secundarios del Eje Y
add_vertical_axis(
    fig_volat, assets_graph=assets_graph, cols_graph=cols_graph, title_primary=eje_y_titulo_1, color_primary=None, title_secondary=eje_y_titulo_2, color_secondary=get_color(1))

set_bloomberg_dark(fig_volat)
fig_volat.show()


## 5.3 Histograma

In [ ]:
datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]

periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
graf_titulo = f'Distribución de Retornos por Activo | Período {periodo}'
eje_x_titulo = f'Retornos %'
eje_y_titulo_1 = f'Frecuencia Observada'

all_data = datos.to_numpy().flatten()
_, bin_edges = np.histogram(all_data, bins=50, density=True)
bin_width = bin_edges[1] - bin_edges[0]  # Ancho de bin

assets_graph = len(datos.columns)
rows_graph = (assets_graph // 2) + (assets_graph % 2)  # 2 columnas por fila
cols_graph = 2


fig_hist = make_subplots(
	rows=rows_graph, cols=cols_graph, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.09, horizontal_spacing=0.10, subplot_titles=[f"<b>{col}</b>" for col in datos.columns]
)

for i, col in enumerate(datos.columns):
	rows_subplot = (i // cols_graph) + 1
	cols_subplot = (i % cols_graph) + 1
	hist_values, _ = np.histogram(datos[col].to_numpy(), bins=bin_edges, density=True)
	data_col = datos[col].to_numpy()

	fig_hist.add_trace(go.Bar(
		x=bin_edges[:-1], y=hist_values, width=bin_width, marker=dict(opacity=0.7, color=get_color(i)), name=col, showlegend=True), 
		row=rows_subplot, col=cols_subplot)

	# KDE (Curva de Densidad)
	kde = gaussian_kde(data_col)
	x_kde = np.linspace(bin_edges.min(), bin_edges.max(), 200)  # Valores X
	y_kde = kde(x_kde)  # Evaluar KDE en esos valores
	fig_hist.add_trace(go.Scatter(
		x=x_kde, y=y_kde, mode='lines', line=dict(color=get_color(7), width=2), name=f"KDE {col}", showlegend=False), 
		row=rows_subplot, col=cols_subplot)
	
	# Ejes X
	fig_hist.update_xaxes(
		showticklabels=True, tickformat=".0%", dtick=0.05, row=rows_subplot, col=cols_subplot)
	
	# # Ejes Y primario
	fig_hist.update_yaxes(
		showticklabels=True, side='left', row=rows_subplot, col=cols_subplot)


# Títulos de ejes X
for col_id in range(1, cols_graph + 1):
	fig_hist.update_xaxes(title=dict(text=eje_x_titulo, standoff=20), row=rows_graph, col=col_id)

# Títulos de ejes Y
for row_id in range(1, rows_graph + 1):
	fig_hist.update_yaxes(title=dict(text=eje_y_titulo_1, standoff=20), row=row_id, col=1)

fig_hist.update_layout(
	width=950, height=305*rows_graph, margin=dict(l=60, r=60, t=90, b=70),
	title=dict(text=graf_titulo, x=0.5, y=0.97),
	legend=dict(bordercolor="black", borderwidth=1, x=0.99, y=0.99, xanchor="right", yanchor="top"))

set_bloomberg_dark(fig_hist)
fig_hist.show()


## 5.4 Scatter Matrix

In [ ]:
datos = returns
fecha_inicio, fecha_fin = datos.index[[0, -1]]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

graf_titulo = f'Pairplot: Histograma de Retornos y gráficos de Dispersión | Período {periodo}'

assets = datos.columns

assets_graph = len(datos.columns)
rows_graph = assets_graph
cols_graph = assets_graph

bins_hist = 20
x_min, x_max = datos.min().min(), datos.max().max()
y_max_hist = max(np.histogram(datos[asset], bins=bins_hist, density=True)[0].max() for asset in datos.columns)

fig_pairplot = make_subplots(
	rows=assets_graph, cols=assets_graph, shared_xaxes=True, shared_yaxes=False, horizontal_spacing=0.03, vertical_spacing=0.02)

def actualizar_ejes(row, col, x_range, y_range, is_histogram=False):
	show_x = row == assets_graph  # Mostrar solo en la última fila
	show_y = col == 1 or is_histogram  # Mostrar en la primera columna o si es un histograma

	fig_pairplot.update_xaxes(
		range=x_range, tickformat=".0%", 
		showticklabels=show_x, row=row, col=col)

	fig_pairplot.update_yaxes(
		range=y_range, tickformat=".0f" if is_histogram else ".0%", showticklabels=show_y, row=row, col=col)


for i, asset_x in enumerate(assets):
	for j, asset_y in enumerate(assets):
		row, col = i + 1, j + 1

		if i == j:  # Histograma
			hist_values, bin_edges = np.histogram(datos[asset_x], bins=bins_hist, density=True)
			bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2

			fig_pairplot.add_trace(go.Bar(x=bin_centers, y=hist_values, marker=dict(opacity=1, color=get_color(i)), showlegend=False), row=row, col=col)
			actualizar_ejes(row, col, [x_min, x_max], [0, y_max_hist * 1.1], is_histogram=True)

		else:  # Gráfico de dispersión
			fig_pairplot.add_trace(go.Scatter(x=datos[asset_x], y=datos[asset_y], mode='markers', marker=dict(size=3, opacity=0.4, color=get_color(i)), showlegend=False), row=row, col=col)
			actualizar_ejes(row, col, [x_min, x_max], [x_min, x_max])


fig_pairplot.update_layout(
	width=1000, height=205*rows_graph, margin=dict(l=60, r=40, t=110, b=60), autosize=True, 
	title=dict(text=graf_titulo, x=0.5, y=0.98))

# Nombres de los activos en la parte superior y en la primera columna
for i, asset in enumerate(assets):
	fig_pairplot.update_xaxes(title_text=f"<b>{asset}</b>", row=1, col=i+1, title_standoff=15, side='top')
	fig_pairplot.update_yaxes(title_text=f"<b>{asset}</b>", row=i+1, col=1)


set_bloomberg_dark(fig_pairplot)
fig_pairplot.show()


## 5.5 Matriz de Correlaciones

In [ ]:
correlations = returns.corr().round(2)
correlations

In [ ]:
datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]

periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
graf_titulo = f'Matriz de Correlaciones | Período {periodo}'
eje_x_titulo = f'Activo'
eje_y_titulo_1 = f'Activo'

correlations = datos.corr().round(2)
tickers = correlations.index.tolist()
correlation_values = correlations.values

font_size_values = 16  # Tamaño de la fuente de los valores en la matriz
label_padding = 10     # Distancia entre los rótulos y la matriz

colorscale = get_colorscale(5)

fig_corr = go.Figure()
fig_corr.add_trace(
	go.Heatmap(
		z=correlation_values, x=tickers, y=tickers, colorscale=colorscale, zmin=-1, zmax=1,
		colorbar=dict(title="Correlación"),
		text=[[f"{val:.2f}" for val in row] for row in correlation_values],
		texttemplate="<span style='font-size:{}px'>%{{text}}</span>".format(font_size_values),
		hovertemplate="<b>%{x}</b> vs <b>%{y}</b>: %{z:.2f}<extra></extra>",
	)
)

fig_corr.update_layout(
	width=700, height=600, margin=dict(l=80, r=40, t=105, b=40),
    title=dict(text=graf_titulo, x=0.5, y=0.97),
    xaxis=dict(title=eje_x_titulo, side="top", title_standoff=label_padding),
    yaxis=dict(title=eje_y_titulo_1, title_standoff=label_padding, autorange="reversed",)
)

set_bloomberg_dark(fig_corr)
# set_bloomberg_light(fig_corr)
fig_corr.show()

# 4. Performance metrics

In [ ]:
ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
end_time = time.time()
execution_time = end_time - star_time

print(f'Correctly executed. Date: {ahora}.')
print(f'\nExecution time: {round(execution_time, 2)} seconds.')